In [34]:
import pandas as pd
import utils.get_datasets as get_datasets
import utils.config as config

In [3]:
# how to get files
# icustays = get_icu_files('icustays.csv.gz')
d_icd_diagnoses = get_datasets.get_hosp_files('d_icd_diagnoses.csv.gz')
icd_diagnoses = get_datasets.get_hosp_files('diagnoses_icd.csv.gz')

print(d_icd_diagnoses, icd_diagnoses)

       icd_code  icd_version  \
0          0010            9   
1          0011            9   
2          0019            9   
3          0020            9   
4          0021            9   
...         ...          ...   
109770     Z992           10   
109771     Z993           10   
109772     Z998           10   
109773    Z9981           10   
109774    Z9989           10   

                                               long_title  
0                          Cholera due to vibrio cholerae  
1                   Cholera due to vibrio cholerae el tor  
2                                    Cholera, unspecified  
3                                           Typhoid fever  
4                                     Paratyphoid fever A  
...                                                   ...  
109770                       Dependence on renal dialysis  
109771                           Dependence on wheelchair  
109772  Dependence on other enabling machines and devices  
109773         

In [4]:
# how to join dataframe
# example - inner join with icd_code (hosp_data)
new_df = pd.merge(left=icd_diagnoses, right=d_icd_diagnoses, how='inner', on='icd_code')
print(new_df)

         subject_id   hadm_id  seq_num icd_code  icd_version_x  icd_version_y  \
0          10000032  22595853        1     5723              9              9   
1          10000826  20032235        4     5723              9              9   
2          10000826  28289260        1     5723              9              9   
3          10005866  26158160        4     5723              9              9   
4          10008924  23676183        7     5723              9              9   
...             ...       ...      ...      ...            ...            ...   
4860275    19990427  29695607       24  T24011A             10             10   
4860276    19996016  28015466        4   O30093             10             10   
4860277    19996783  25894657       20  H353131             10             10   
4860278    19997062  20096107        1    K8036             10             10   
4860279    19999043  23037011       10  O359XX2             10             10   

                           

In [36]:
args = utils.config.get_arguments()
print(args)

AttributeError: module 'utils.config' has no attribute 'get_arguments'